# Aggregate Window Functions

We have already met many of these aggregates previously when introducing GROUP BY. However, with GROUP BY we are aggregating several records into a single returned value, whereas with Window Functions we can add range based aggregates to each individual record, therefore maintaining the volume of data we return each time.

We also, in this section, introduce an additional syntax to correctly 'frame' our data and ensure correct results from our Window Functions.  

This syntax is a lot more complex than most SQL and will likely be the hardest you will have to learn as a Developer.  Luckily the syntax itself can be easily cut and pasted between queries, it's more the usage that is crtitical to learn and understand.

To try and introduce ourselves to this syntax and way of thinking, we will start with MIN.

#### MIN

We know from our work with GROUP BY that MIN returns us the minimum value within a dataset:

In [ ]:
select min(saleAmount)
from JupyterDatabase.sales.salesOrderHeaderWindow;

As expected this returns a single record based on the dataset as a whole, showing us the minimum value.

However, when used as a Window Function we can obtain this same value, but without compressing our dataset.  Visually it looks as follows:

![Min](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Min.jpg)

Let's do the same with our sales.salesOrderHeaderWindow table:

In [ ]:
select min(saleAmount) over(order by saleAmount), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

As you can see, we obtain the very same minimum value, but this time placed against every row in our table rather than compressed to a single record output.

We can also PARTITION BY salesPersonID to provide the minimum saleAmount for each SalesPersonID:

In [ ]:
select min(saleAmount) over(partition by salesPersonID), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This is all working wonderfully so far, therefore where is the concern over 'incorrect' results and the need for additional syntax?

To find out, let's run the following piece of code and see what results are returned:

In [ ]:
select min(saleAmount) over(order by saleAmount desc), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This suddenly looks very wrong indeed.  But why?

**Framing our Windows**

This actually boils down to the direction in which we ordered our data within the Window Function and the order in which the Window Function operates.

Let's look at a visual to explain what's happening:

![MinSeen](https://www.sqlworld.co.uk/images/Blogs/SQLServer/MinSeen.jpg)

As the visual shows us, when we forcibly order the data that is the order in which the Window Function 'sees' the data and therefore it can dramatically change the output shown and seemingly produce 'incorrect' data when actually it is working as defined.

In the ASC ordered data our Window Function 'sees' the lowest number first and therefore reports it back.  The next number is higher and so it still reports the original, lowest, value.

In the DESC ordered data our Window Function 'sees' the highest number first but as it has seen no other numbers it marks it as the lowest.  It then 'sees' the next number which is lower and so it reports that as the new MIN value (it cannot correct backwards once it has moved on), and so on.  Therefore, with the data being highest to lowest, the MIN value is never correct until we reach the last record in the dataset.

In order to avoid this issue we can either amend our ORDER BY within the OVER clause or, as this is not always necessary and may not always achieve the results we wish, we can utilise some additional syntax reserved solely for defining a 'frame' for our Window Functions to operate in.

We will dive right into the syntax and then explain it immediately after (I have spaced out the code to add clarity):

In [ ]:
select min(saleAmount)
		over
		(
			order by saleAmount desc
			rows between current row and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Bingo. This still specifies DESC for our ordering and yet returns the correct results. We can also clearly see why as we have added an additional piece of guidance for the Window Function to consider.

As Window Functions, by default, only look backwards at rows they have passed, in this case we need to tell it instead to look forwards to the end of the dataset.

There are a few versions of this additional syntax which we can cut and paste into our queries as we see fit (hence at the beginning of this section I mentioned it's complex syntax but once understood can be copied and pasted):

To always look forward in the dataset:

`rows between current row and unbounded following`

To always look backwards in the dataset:

`rows between unbounded preceding and current row`

To consider the entire dataset backwards and forwards:

`rows between unbounded preceding and unbounded following`

It is this additional syntax which is considered the 'frame' for our Windows. It defines the selection of records on which we wish our Window Functions to consider.

We shall use these more and more from now on so that you can really get the feel for them.

#### MAX

Now that we have seen MIN working, it should come as no surprise that MAX is identical in all ways, except that it will naturally give us the expected results when the data is ordered DESC (as it will therefore see the largest value first).

![Max](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Max.jpg)

However, instead of relying on ordering we shall be specific with our framing in order to ensure we know what we are getting, using unbounded preceding and following to ensure the whole 'frame' is considered when looking for MAX:

In [ ]:
select max(saleAmount)
		over
		(
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This gives us the exact result we expected.

We can also add partitioning on salesPersonID to obtain the MAX saleAmount per salesPersonID:

In [ ]:
select max(saleAmount)
		over
		(
			partition by salesPersonID
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Despite the complex syntax, framing is actually simple to use once understood and can simply be copied and pasted into place as required.

#### COUNT

A count of records is, once again, identical to MIN and MAX with its usage, requiring a frame in order to provide a correct row count for our results.

![Count](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Count1.jpg)

Using the same examples as above with MAX we can see how this works:

In [ ]:
select count(saleAmount)
		over
		(
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

And utilising a partition on SalesPersonID:

In [ ]:
select count(saleAmount)
		over
		(
			partition by salesPersonID
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

#### AVG

Average works in the same way in regard to framing the data however, in this case there are genuinely good reasons why we may not frame the data at all, instead opting to leave the row by row value changes in place.

Firstly we will look at the framed version which, as we would expect, will produce an average across our frame and then place this value against each record in our dataset:

In [ ]:
select avg(saleAmount)
		over
		(
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This will work for a partitioned dataset as well:

In [ ]:
select avg(saleAmount)
		over
		(
			partition by salesPersonID
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

However, where AVG can shine is that we, as we know from MIN and MAX above, can work on a row by row basis, dealing with only the data that we have seen so far as the Window Function travels through the records.

Whereas with MIN and MAX this is not very useful, with AVG we can actually achieve a Rolling Average:

![Average](https://www.sqlworld.co.uk/images/Blogs/SQLServer/avg.jpg)

As shown in the visualisation above, we can generate a rolling average:
* Average of 100 is 100
* (100 + 40) / 2 = 70
* (100 + 40 + 60) / 3 = 66

This can be vital information in a reporting environment.

Let's perform this same action using our sales.salesOrderHeaderWindow table to produce a rolling average saleAmount for each SalesPersonID:

In [ ]:
select avg(saleAmount)
		over
		(
			partition by salesPersonID
			order by orderDate
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This is very good for flattening spikes within data when producing long term trends, for example.

#### SUM

Finally we come to SUM which acts exactly the same way as AVG except that AVG provides a rolling average whereas SUM performs a cumulative total.

Once again we have the option to obtain our SUM over the entire frame as one:

In [ ]:
select sum(saleAmount)
		over
		(
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Including at the partition level:

In [ ]:
select sum(saleAmount)
		over
		(
			partition by salesPersonID
			order by saleAmount
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

However, as mentioned, we also have the ability to use it to produce a cumulative (running) total:

![Sum](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Sum.jpg)

We can demonstrate this using our sales.salesOrderHeaderWindow table as follows, including a partition on SalesPersonID:

In [ ]:
select sum(saleAmount)
		over
		(
			partition by salesPersonID
			order by orderDate
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This can be a very powerful tool within a reporting environment, obtained with minimal code and effort.